In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 1000
    m.NumPaths = 2
    m.Add(sdk.IndependentGaussian())
    return m

In [3]:
time_horizon = 2.5
model = BuildModel_Simple()
# model.evaluations.append(sdk.EvaluationPoint(0,time_horizon/2))
model.evaluations.append(sdk.EvaluationPoint(0,time_horizon))

### Short rate process
2% constant interest rate, we set it as brownian motion process with drift=0 and diffusion=0.

In [4]:
short_rate_start = 0.02
short_rate_process = model.Add(sdk.BrownianMotion(
    start     = short_rate_start,
    drift     = 0.0,
    diffusion = 0.0,
    title     = "short rate"
))
print(short_rate_process.GetStateNumber())

0


In [5]:
discount_process = model.Add(sdk.ZeroCouponBond(
    underlying = short_rate_process.GetStateNumber(),
    start      = 1,
    title      = "discount factor"
))
print(discount_process.GetStateNumber())

1


discount_process.GetEquationNumber()

In [6]:
notional = 1000
points = 3
steps_dt = model.TimeSteps/points
t = [int(steps_dt*(i+1))-1 for i in range(points)]
# print('t = ',t)
fixed_leg_process = model.Add(sdk.SwapFixedLeg(
    ref = discount_process.GetStateNumber(),
    notional = notional,
    t = t,
    title = "fixed leg"
))
print(discount_process.GetStateNumber())

AttributeError: module 'mcsdk' has no attribute 'SwapFixedLeg'

In [ ]:
fixed_leg_process.args

In [ ]:
model.json()

In [ ]:
results = run (model)
results.df()

In [ ]:
from math import *
fixed_leg_value = 0
for i in range(points):
    df = exp(-short_rate_start*time_horizon*(i+1)/points)
    print(df)
    fixed_leg_value += df*notional
print(f'fixed_leg_value={fixed_leg_value}')
    


$$
\sum_{i=1}^m P(t_0,t_i) N_i
$$
- $t_0$ is the _current_ time poiunt
- $m$ the number of time intrervals of the leg ($m>0$)
- $N_i$ the notional value at time point $t_i$

\begin{eqnarray}
\sum_{i=1}^{n}
\end{eqnarray}

## Two passes MonteCarlo
Consider a $P(T,T+\tau)$ discount factor from $T$ to $T+\tau$, which is observed at time $T$. If
$$
P(T,T+\tau) = \exp\left[-\int_{T}^{T+\tau} r(t)\cdot dt \right]
$$
then it can be computed numerically for any reasonable short rate process $r(t)$. With the MonteCarlo simulation the computation will be completed at time $T+\tau$, so if the discounted factor was needed at $T$, then the second pass is required.

To handle this sutuation I need:
- two passes process
- both processes must be started using the same initial random number seed value
- in the first pass a process $S$ (like discounte factor) will be _recorded_ at the user provided $t_i$ points, thus a set of $(S_i,t_i)$ points will be available.
- $(S_i,t_i)$ will be used in the second pass to finalyze the computations

